# Extraction Chain Rule with LangChain and ChatGroq

In this notebook, we demonstrate how to build an extraction chain rule using LangChain and integrate it with ChatGroq for extracting structured information from unstructured text.

## The Schema
First, we need to describe what information we want to extract from the text.

We'll use Pydantic to define an example schema to extract personal information.

In [47]:
from typing import Optional

from pydantic import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    # ^ Doc-string for the entity Person.
    # This doc-string is sent to the LLM as the description of the schema Person,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    name: Optional[str] = Field(default=None, description="The name of the person")
    hair_color: Optional[str] = Field(
        default=None, description="The color of the person's hair if known"
    )
    height_in_meters: Optional[str] = Field(
        default=None, description="Height measured in meters"
    )

Here are two important tips for creating schemas:

### 1. Add Descriptions for Attributes and the Schema:

Clearly document each attribute and the overall schema. This helps the LLM understand what information to extract, making the results more accurate.

### 2. Avoid Forcing the LLM to Guess: 

Use Optional for attributes, so the LLM can return None if it doesn't have the answer. This prevents it from making up information.

## The Extractor
Let's create an information extractor using the schema we defined above.

In [48]:
from typing import Optional

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel, Field

# Define a custom prompt to provide instructions and any additional context.
# 1) You can add examples into the prompt template to improve extraction quality
# 2) Introduce additional parameters to take context into account (e.g., include metadata
#    about the document from which the text was extracted.)
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        # Please see the how-to about improving performance with
        # reference examples.
        # MessagesPlaceholder('examples'),
        ("human", "{text}"),
    ]
)

We need to use a model that supports function/tool calling.

In [49]:
import getpass
import os

if not os.environ.get("GROQ_API_KEY"):
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")

In [50]:
structured_llm = llm.with_structured_output(schema=Person)

In [51]:
text = "Alan Smith is 6 feet tall and has blond hair."
prompt = prompt_template.invoke({"text": text})
structured_llm.invoke(prompt)

Person(name='Alan Smith', hair_color='blond', height_in_meters='1.83')

## Multiple Entities
In most cases, you should be extracting a list of entities rather than a single entity.

This can be easily achieved using pydantic by nesting models inside one another.

In [52]:
from typing import List, Optional

from pydantic import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    # ^ Doc-string for the entity Person.
    # This doc-string is sent to the LLM as the description of the schema Person,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    name: Optional[str] = Field(default=None, description="The name of the person")
    hair_color: Optional[str] = Field(
        default=None, description="The color of the person's hair if known"
    )
    height_in_meters: Optional[str] = Field(
        default=None, description="Height measured in meters"
    )


class Data(BaseModel):
    """Extracted data about people."""

    # Creates a model so that we can extract multiple entities.
    people: List[Person]

In [53]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        # Please see the how-to about improving performance with
        # reference examples.
        # MessagesPlaceholder('examples'),
        ("human", "{text}"),
    ]
)

In [54]:
structured_llm = llm.with_structured_output(schema=Data)
text = "My name is Jeff, my hair is black and i am 6 feet tall. Anna has the same color hair as me."
prompt = prompt_template.invoke({"text": text})
structured_llm.invoke(prompt)

Data(people=[Person(name='Jeff', hair_color='black', height_in_meters='1.83'), Person(name='Anna', hair_color='black', height_in_meters=None)])

In [55]:
messages = [
    {"role": "user", "content": "2 🦜 2"},
    {"role": "assistant", "content": "4"},
    {"role": "user", "content": "2 🦜 3"},
    {"role": "assistant", "content": "5"},
    {"role": "user", "content": "3 🦜 4"},
]

response = llm.invoke(messages)
print(response.content)

7


The pattern in the messages list is:

* The user sends a mathematical expression involving the symbol 🦜 (likely representing a placeholder or operation).

* The assistant provides the result, presumably performing some operation (like addition, subtraction, or another type of operation).


The interaction appears to involve simple arithmetic, with the symbol 🦜 acting as a separator or operator.

#### User's Input: "2 🦜 2"
- The assistant responds with **"4"**, assuming that 🦜 acts as a placeholder for addition (i.e., 2 + 2).

#### User's Input: "2 🦜 3"
- The assistant responds with **"5"**, interpreting 🦜 as addition (2 + 3).

#### User's Input: "3 🦜 4"
- The assistant responds with **"7"**, performing 3 + 4.


LangChain includes a utility function tool_example_to_messages that will generate a valid sequence for most model providers. It simplifies the generation of structured few-shot examples by just requiring Pydantic representations of the corresponding tool calls.

Let's try this out. We can convert pairs of input strings and desired Pydantic objects to a sequence of messages that can be provided to a chat model. Under the hood, LangChain will format the tool calls to each provider's required format.

In [56]:
from langchain_core.utils.function_calling import tool_example_to_messages

examples = [
    (
        "The oceana is vast and blue. It's more than 20,000 feet deep.",
        Data(people=[]),
    ),
    (
        "Fairy traveled far from France to Spain.",
        Data(people=[Person(name="Fairy", height_in_meters=None, hair_color=None)]),
    ),
]


messages = []

for txt, tool_call in examples:
    if tool_call.people:
        # This final message is optional for some providers
        ai_response = "Detected people."
    else:
        ai_response = "Detected no people."
    messages.extend(tool_example_to_messages(txt, [tool_call], ai_response=ai_response))

In [57]:
for message in messages:
    message.pretty_print()

================================ Human Message =================================

The oceana is vast and blue. It's more than 20,000 feet deep.
================================== Ai Message ==================================
Tool Calls:
  Data (9e4f9809-bb10-4e38-be82-221a80d515e5)
 Call ID: 9e4f9809-bb10-4e38-be82-221a80d515e5
  Args:
    people: []
================================= Tool Message =================================

You have correctly called this tool.
================================== Ai Message ==================================

Detected no people.
================================ Human Message =================================

Fairy traveled far from France to Spain.
================================== Ai Message ==================================
Tool Calls:
  Data (891626a5-d0ae-4434-a940-02c272a4d44e)
 Call ID: 891626a5-d0ae-4434-a940-02c272a4d44e
  Args:
    people: [{'name': 'Fairy', 'hair_color': None, 'height_in_meters': None}]
===============================

Let's compare performance with and without these messages. For example, let's pass a message for which we intend no people to be extracted:

In [64]:
message_no_extraction = {
    "role": "user",
    "content": "The solar system is large, but earth has only 1 moon.",
}

structured_llm = llm.with_structured_output(schema=Data)
structured_llm.invoke([message_no_extraction])

Data(people=[Person(name='Earth', hair_color=None, height_in_meters=None)])

In this example, the model is liable to erroneously generate records of people.

Because our few-shot examples contain examples of "negatives", we encourage the model to behave correctly in this case:

In [65]:
structured_llm.invoke(messages + [message_no_extraction])

Data(people=[])

## Second Example
Pydantic model

In [59]:
from typing import Optional
from pydantic import BaseModel, Field
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
import json


# Define the Product schema using Pydantic
class Product(BaseModel):
    """Information about a product."""

    # Doc-string helps the LLM understand the purpose of extraction
    product_name: Optional[str] = Field(
        default=None, description="The name of the product"
    )
    price: Optional[str] = Field(
        default=None, description="The price of the product in USD"
    )
    availability: Optional[str] = Field(
        default=None,
        description="The availability status of the product (e.g., 'In stock', 'Out of stock')",
    )

In [60]:
# Define the prompt template for extraction
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        ("human", "{text}"),
    ]
)

In [61]:
# Set up the LLM
llm = ChatGroq(model="llama3-8b-8192")  # Replace with your LLM provider and model


In [62]:
# Create the extraction chain
input_text = """
The latest gadget, the SmartWidget 3000, is now available for $299.99. It is currently in stock and ready for delivery.
"""
# Validate the extracted data against the Product schema
try:
    structured_llm = llm.with_structured_output(schema=Product)
    prompt = prompt_template.invoke({"text": input_text})
    response = structured_llm.invoke(prompt)
    print(response)
except Exception as e:
    print("Validation Error:", str(e))

product_name='SmartWidget 3000' price='$299.99' availability='In stock'


## Difference: Classification vs Extraction

### Classification
#### Goal: 
Assign predefined categories or labels to an input based on its content.

#### Key Characteristics:
- **Input**: Typically unstructured text or data.
- **Output**: One or more predefined categories (labels).
- **Prompt**: Focuses on guiding the model to identify and assign a label (e.g., sentiment, topic, spam vs. not spam).
- **Schema**: No complex schema is required; the output is often a simple string or list of labels.

#### Use Case: 
Classifying customer feedback into sentiment categories like "Positive," "Negative," or "Neutral."

---



### Extraction
#### Goal: 
Extract specific structured information or attributes from unstructured data.

#### Key Characteristics:
- **Input**: Typically unstructured text containing detailed information.
- **Output**: A structured schema with extracted attributes, validated using tools like Pydantic.
- **Prompt**: Focuses on instructing the model to extract specific fields or entities.
- **Schema**: Requires a predefined schema (e.g., using Pydantic) to validate and structure the output.

#### Use Case: 
Extracting details like name, email, and phone number from a customer query.
